In [ ]:
from typing import Tuple
import numpy as np
import mediapy
from tqdm import tqdm
import dataclasses
import jax
import jax.numpy as jnp
from waymax import config as _config
from waymax import dataloader
from waymax import datatypes
from waymax import visualization
import cv2

max_num_objects = 32
config = dataclasses.replace(
    _config.WOD_1_1_0_TRAINING,
    max_num_objects=max_num_objects,
    max_num_rg_points=30000,
    path="./data/training_tfexample.tfrecord@5",
)
data_iter = dataloader.simulator_state_generator(config=config)
scenario = next(data_iter)
scenario = next(data_iter)
scenario = next(data_iter)
scenario = next(data_iter)
scenario = next(data_iter)


img = visualization.plot_simulator_state(scenario, use_log_traj=True)
imgs = []
ray_visualization = []
state = scenario


def ray_segment_intersection(
    ray_angle: jax.Array, start_points: jax.Array, segment_dirs: jax.Array
) -> jax.Array:
    """
    Calculate the intersection distances between a ray and line segments.

    Args:
        ray_angle: The angle of the ray (in radians).
        start_points: Array of shape (N, 2) for segment start points (x,y).
        segment_dirs: Array of shape (N, 2) for segment directions (dx, dy).

    Returns:
        Array of distances from origin to intersections. Returns 100.0 if no intersection.
    """
    # Calculate ray direction
    ray_dir_x = jnp.sin(ray_angle)
    ray_dir_y = jnp.cos(ray_angle)

    # Calculate segment direction
    segment_dir_x = segment_dirs[:, 0]
    segment_dir_y = segment_dirs[:, 1]

    # Calculate determinant for intersection test
    det = segment_dir_x * ray_dir_y - segment_dir_y * ray_dir_x

    # Avoid division by zero for parallel lines
    is_parallel = jnp.abs(det) < 1e-8
    det = jnp.where(is_parallel, 1.0, det)  # Avoid division by zero

    # Calculate t1 and t2 parameters
    t1 = (start_points[:, 0] * ray_dir_y - start_points[:, 1] * ray_dir_x) / det
    t2 = (start_points[:, 0] * segment_dir_y - start_points[:, 1] * segment_dir_x) / det

    # Check if intersection is within segment (0 <= t1 <= 1) and ray (t2 >= 0)
    valid_t1 = (t1 >= 0.0) & (t1 <= 1.0)
    valid_t2 = t2 >= 0.0

    # Combine intersection validity checks
    valid_intersection = valid_t1 & valid_t2 & ~is_parallel

    # Calculate intersection points and distances
    ix = start_points[:, 0] + t1 * segment_dir_x
    iy = start_points[:, 1] + t1 * segment_dir_y

    # Distance from origin to intersection point
    distances = jnp.sqrt(ix**2 + iy**2)

    # Make sure distances over 100 are not valid
    valid_intersection = valid_intersection & (distances < 100.0)
    
    # Return distance if valid intersection, otherwise 100.0
    return jnp.where(valid_intersection, distances, 100.0)


def find_closest_distance(
    i, initval: Tuple[jax.Array, datatypes.RoadgraphPoints, jax.Array]
):
    circogram, rg_points, ray_angles = initval
    ray_angle = ray_angles[i]

    # Only consider valid points
    candidate_points = rg_points.valid

    # Only consider road edge points
    candidate_points = candidate_points & (
        (rg_points.types == datatypes.MapElementIds.ROAD_EDGE_BOUNDARY)
        | (rg_points.types == datatypes.MapElementIds.ROAD_EDGE_MEDIAN)
        | (rg_points.types == datatypes.MapElementIds.ROAD_EDGE_UNKNOWN)
    )

    # If no valid points, return the default distance
    has_valid = jnp.any(candidate_points)

    # Create line segments from roadgraph points
    starting_points = jnp.stack([rg_points.x, rg_points.y], axis=1)
    dir_xy = jnp.stack([rg_points.dir_x, rg_points.dir_y], axis=1)

    # Calculate intersection distances
    intersection_distances = ray_segment_intersection(
        ray_angle, starting_points, dir_xy
    )
    masked_distances = jnp.where(candidate_points, intersection_distances, 100.0)

    # Find minimum distance among valid points
    min_distance = jnp.min(masked_distances)

    # Update only the i-th element and return the whole array
    circogram = circogram.at[i].set(jnp.where(has_valid, min_distance, circogram[i]))
    return circogram, rg_points, ray_angles


def create_circogram(observation: datatypes.Observation, num_rays: int) -> jax.Array:
    ray_angles = jnp.linspace(0, 2 * jnp.pi, num_rays, endpoint=False)
    rg_points = observation.roadgraph_static_points
    # For each ray angle, find the closest intersection
    circogram = jnp.full(num_rays, 100.0)  # Default max distance
    (
        circogram,
        _,
        _,
    ) = jax.lax.fori_loop(
        0,
        num_rays,
        find_closest_distance,
        (circogram, rg_points, ray_angles),
    )
    return circogram



jit_step = jax.jit(datatypes.update_state_by_log, static_argnums=(1,))
jit_observed = jax.jit(datatypes.sdc_observation_from_state, static_argnums=(1,2))
jit_create_circogram = jax.jit(create_circogram, static_argnums=(1))
for _ in tqdm(range(scenario.remaining_timesteps)):
    state = jit_step(state, num_steps=1)
    #imgs.append(visualization.plot_simulator_state(state, use_log_traj=True))
    imgs.append(
        visualization.plot_observation(
            jit_observed(state, roadgraph_top_k=2000),
            obj_idx=0
        )
    )

    num_rays = 64
    ray_angles = jnp.linspace(-jnp.pi, jnp.pi, num_rays, endpoint=False)
    observation = jit_observed(state, roadgraph_top_k=2000)
    circogram = jit_create_circogram(observation, num_rays)
    

  # Create a visualization of ray distances
    small_size = 500  # Size of the visualization image
    scale_factor = 5.0  # Scale factor for distance visualization
    x = small_size // 2  # Center of the image
    y = small_size // 2  # Center of the image
    ray_img = np.ones((small_size, small_size, 3), dtype=np.uint8) * 255  # White background
    center = (x, y)  # Center of the image

    # Draw rays
    for i in range(num_rays):
        # Calculate endpoint based on ray angle and distance
        angle = ray_angles[i]
        dist = circogram[i]

        # Scale the distance for visualization
        scaled_dist = dist * scale_factor

        # Calculate endpoint
        end_x = int(center[0] + np.sin(angle) * scaled_dist)
        end_y = int(center[1] + np.cos(angle) * scaled_dist)

        # Draw a line from center to endpoint
        cv2.line(ray_img, center, (end_x, end_y), (0, 0, 255), 1)  # Red lines

    # Add a circle for the SDC
    cv2.circle(ray_img, center, 5, (0, 255, 0), -1)  # Green circle for SDC

    # Flip the ray image upside down to match conventional coordinate system
    ray_img = cv2.flip(ray_img, 0)

    # Draw a reference circle at maximum range
    max_range_radius = int(100.0 * scale_factor)  # 100 meters reference
    cv2.circle(ray_img, center, max_range_radius, (200, 200, 200), 1)  # Gray circle
    # Add distance markers
    for dist in [20, 40, 60, 80]:
        radius = int(dist * scale_factor)
        cv2.circle(ray_img, center, radius, (220, 220, 220), 1, cv2.LINE_AA)
        # Add text label
        text_pos = (center[0] + 5, center[1] - radius - 5)
        cv2.putText(ray_img, f"{dist}m", text_pos, cv2.FONT_HERSHEY_SIMPLEX, 
                    0.4, (100, 100, 100), 1, cv2.LINE_AA)

    # Add to ray_visualization array
    ray_visualization.append(ray_img)
mediapy.show_video(imgs, fps=10)
mediapy.show_video(ray_visualization, fps=10)



100%|██████████| 90/90 [00:11<00:00,  8.06it/s]


This browser does not support the video tag.


This browser does not support the video tag.
